## Практическая работа "TTS Системы"
### Выполнил: Сидельников Дэвид Константинович, группа КЭ-228

In [14]:
import base64
import requests

def get_tts_google(api_key, text):
    url = "https://text-to-speech-neural-google.p.rapidapi.com/generateAudioFiles"
    
    # Формирование данных для отправки запроса
    payload = {
        "audioFormat": "mp3",  # Формат аудиофайла MP3
        "paragraphChunks": [text],  # Текст для озвучивания
        "voiceParams": {
            "name": "Wavenet-B",  # Имя выбранного голоса
            "engine": "google",  # Движок синтеза речи
            "languageCode": "ru-RU"  # Код языка (в данном случае русский)
        }
    }
    
    # Заголовки, включая API-ключ
    headers = {
        "Content-Type": "application/json",
        "x-rapidapi-host": "text-to-speech-neural-google.p.rapidapi.com",
        "x-rapidapi-key": api_key  # Здесь должен быть твой API-ключ
    }
    
    # Выполняем запрос
    response = requests.post(url, json=payload, headers=headers)
    
    # Проверяем успешность запроса
    if response.status_code == 200:
        response_data = response.json()
        
        # Проверяем наличие поля "audioStream"
        if response_data and "audioStream" in response_data:
            # Аудио возвращается в base64 формате, декодируем его
            audio_content = base64.b64decode(response_data["audioStream"])
            
            # Сохраняем результат в MP3 файл
            with open("output_google_tts.mp3", "wb") as f:
                f.write(audio_content)
            print("Озвучка успешно сгенерирована и сохранена как output_google_tts.mp3")
        else:
            print("Ошибка: не удалось получить аудиофайл.")
    else:
        print(f"Ошибка при обращении к API: {response.status_code}, {response.text}")

# Текст из фильма "Матрица"
text = """Это твой последний шанс. Потом пути назад не будет. Примешь синюю таблетку — и история закончится, 
          ты проснёшься в своей постели и поверишь, во что захочешь. Примешь красную таблетку — останешься 
          в Стране чудес, и я покажу тебе, насколько глубока кроличья нора."""

# Твой API-ключ
api_key = "0c9e7e7d9bmshecbe5afdcb2b873p16539ajsne1e35edcd88e"

# Вызов функции
get_tts_google(api_key, text)


Ответ API: {'audioStream': '...', 'format': 'mp3', 'speechMarks': {'chunks': [{'type': 'sentence', 'start': 0, 'end': 283, 'startTime': 0, 'endTime': 17640, 'value': 'Это твой последний шанс. Потом пути назад не будет. Примешь синюю таблетку — и история закончится,            ты проснёшься в своей постели и поверишь, во что захочешь. Примешь красную таблетку — останешься            в Стране чудес, и я покажу тебе, насколько глубока кроличья нора.', 'chunks': [{'type': 'word', 'start': 0, 'end': 3, 'startTime': 14.999999664723873, 'endTime': 222.99998998641968, 'value': 'Это'}, {'type': 'word', 'start': 4, 'end': 8, 'startTime': 222.99998998641968, 'endTime': 453.041672706604, 'value': 'твой'}, {'type': 'word', 'start': 9, 'end': 18, 'startTime': 453.041672706604, 'endTime': 1011.6250514984131, 'value': 'последний'}, {'type': 'word', 'start': 19, 'end': 23, 'startTime': 1011.6250514984131, 'endTime': 1677.125096321106, 'value': 'шанс'}, {'type': 'word', 'start': 25, 'end': 30, 'startTim

In [19]:
import os
from tortoise.utils.audio import get_voices

# Путь к папке с голосами
voices_dir = os.path.join(os.getcwd(), 'tortoise-tts/tortoise/voices')

# Получаем список доступных голосов
voices = get_voices([voices_dir])

print("Доступные голоса:", voices.keys())


Доступные голоса: dict_keys(['jlaw', 'train_grace', 'train_atkins', 'william', 'train_lescault', 'myself', 'tom', 'snakes', 'tim_reynolds', 'lj', 'train_empire', 'cond_latent_example', 'halle', 'train_dreams', 'daniel', 'train_dotrice', 'deniro', 'applejack', 'weaver', 'train_daws', 'train_mouse', 'geralt', 'rainbow', 'angie', 'emma', 'train_kennard', 'pat2', 'mol', 'pat', 'freeman'])


In [20]:
import sys
import os
import torchaudio  # Для работы с аудиофайлами

# Добавляем путь к папке 'tortoise-tts' в PYTHONPATH
sys.path.append(os.path.join(os.getcwd(), 'tortoise-tts'))

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_voice

def generate_tts(text, voice_name):
    tts = TextToSpeech()

    # Используем выбранный голос
    voice_samples, conditioning_latents = load_voice(voice_name)

    # Генерация речи
    gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, preset='fast')

    # Сохранение результата в файл через torchaudio
    torchaudio.save("output_tortoise.wav", gen.unsqueeze(0), 24000)  # Сохранение с частотой 24000 Hz
    print("Аудиофайл успешно создан: output_tortoise.wav")

if __name__ == "__main__":
    text = """Это твой последний шанс. Потом пути назад не будет. Примешь синюю таблетку — и история закончится, 
             ты проснёшься в своей постели и поверишь, во что захочешь. Примешь красную таблетку — останешься 
             в Стране чудес, и я покажу тебе, насколько глубока кроличья нора."""

    # Используем голос 'freeman'
    generate_tts(text, 'freeman')


Some weights of the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at jbetker/wav2vec2-large-robust-ft-libritts-voxpopuli and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.o

Generating autoregressive samples..


  0%|          | 0/24 [08:39<?, ?it/s]


KeyboardInterrupt: 

: 